In [41]:
import numpy as np
import pandas as pd
import cv2
import PIL
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


In [42]:
dataset_url = '/Users/varunkaleeswaranbalasubramani/Documents/skin-disease-datasaet/train_set copy'

In [43]:
os.listdir(dataset_url)

['.DS_Store',
 'FU-athlete-foot',
 'VI-chickenpox',
 'VI-shingles',
 'FU-nail-fungus',
 'BA-impetigo',
 'PA-cutaneous-larva-migrans',
 'FU-ringworm',
 'BA- cellulitis']

In [44]:
dataset_url

'/Users/varunkaleeswaranbalasubramani/Documents/skin-disease-datasaet/train_set copy'

In [45]:
import pathlib
data_dir=pathlib.Path(dataset_url)
data_dir

PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/skin-disease-datasaet/train_set copy')

In [46]:
cellulitis=list(data_dir.glob('BA- cellulitis/*'))
len(cellulitis)

136

In [9]:
disease_images_train_dic={
    'cellulitis':list(data_dir.glob('BA- cellulitis/*')),
    'impetigo':list(data_dir.glob('BA-impetigo/*')),
    'athlete-foot':list(data_dir.glob('FU-athlete-foot/*')),
    'nail-fungus':list(data_dir.glob('FU-nail-fungus/*')),
    'ringworm':list(data_dir.glob('FU-ringworm/*')),
    'cutaneous-larva-migrans':list(data_dir.glob('PA-cutaneous-larva-migrans/*')),
    'chickenpox':list(data_dir.glob('VI-chickenpox/*')),
    'shingles':list(data_dir.glob('VI-shingles/*')),
#     'normal':list(data_dir.glob('normal/*')),   
}

In [61]:
import cv2
from pathlib import Path


# Define the image enhancement function
def enhance_image(image):
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Convert to YUV color space
    yuv = cv2.cvtColor(blurred, cv2.COLOR_BGR2YUV)
    
    # Apply Laplacian filter for edge enhancement on Y channel
    laplacian = cv2.Laplacian(yuv[:,:,0], cv2.CV_64F)
    
    # Normalize Laplacian for better contrast
    laplacian = cv2.normalize(laplacian, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # Combine Laplacian with original Y channel to enhance edges
    enhanced_y = cv2.addWeighted(yuv[:,:,0], 1.5, laplacian, -0.5, 0)
    
    # Apply histogram equalization to enhance contrast
    enhanced_y = cv2.equalizeHist(enhanced_y)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for better local contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced_y = clahe.apply(enhanced_y)
    
    # Apply unsharp masking for edge and detail enhancement
    blurred = cv2.GaussianBlur(image, (0,0), 3)
    unsharp_mask = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    
    # Merge enhanced Y channel with original U and V channels
    enhanced_yuv = cv2.merge([enhanced_y, yuv[:,:,1], yuv[:,:,2]])
    
    # Convert back to BGR color space
    enhanced_bgr = cv2.cvtColor(enhanced_yuv, cv2.COLOR_YUV2BGR)
    
    return enhanced_bgr

# Define the dictionary containing paths to images for different disease categories
disease_images_train_dic = {
    'cellulitis': list(data_dir.glob('BA- cellulitis/*')),
    'impetigo': list(data_dir.glob('BA-impetigo/*')),
    'athlete-foot': list(data_dir.glob('FU-athlete-foot/*')),
    'nail-fungus': list(data_dir.glob('FU-nail-fungus/*')),
    'ringworm': list(data_dir.glob('FU-ringworm/*')),
    'cutaneous-larva-migrans': list(data_dir.glob('PA-cutaneous-larva-migrans/*')),
    'chickenpox': list(data_dir.glob('VI-chickenpox/*')),
    'shingles': list(data_dir.glob('VI-shingles/*')),
    # 'normal': list(data_dir.glob('normal/*')),
}

# Iterate over each disease category and its corresponding image paths
for disease, image_paths in disease_images_train_dic.items():
    # Create a directory for saving enhanced images if it doesn't exist
    Path("enhanced_images/" + disease).mkdir(parents=True, exist_ok=True)
    
    # Iterate over each image path
    for image_path in image_paths:
        # Read the image
        image = cv2.imread(str(image_path))
        
        # Apply image enhancement
        enhanced_image = enhance_image(image)
        
        # Save the enhanced image to the corresponding folder
        output_path = "enhanced_images/{}/{}".format(disease, image_path.name)
        cv2.imwrite(output_path, enhanced_image)
dataset_url = '/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images'
data_dir=pathlib.Path(dataset_url)
disease_images_train_dic = {
    'cellulitis': list(data_dir.glob('cellulitis/*')),
    'impetigo': list(data_dir.glob('impetigo/*')),
    'athlete-foot': list(data_dir.glob('athlete-foot/*')),
    'nail-fungus': list(data_dir.glob('nail-fungus/*')),
    'ringworm': list(data_dir.glob('ringworm/*')),
    'cutaneous-larva-migrans': list(data_dir.glob('cutaneous-larva-migrans/*')),
    'chickenpox': list(data_dir.glob('chickenpox/*')),
    'shingles': list(data_dir.glob('shingles/*')),
    # 'normal': list(data_dir.glob('normal/*')),
}

In [62]:
# Add this before the loop
print("Cellulitis images:", disease_images_train_dic['cellulitis'])

# Iterate over each disease category and its corresponding image paths
for disease, image_paths in disease_images_train_dic.items():
    # Create a directory for saving enhanced images if it doesn't exist
    Path("enhanced_images/" + disease).mkdir(parents=True, exist_ok=True)
    
    # Iterate over each image path
    for image_path in image_paths:
        # Read the image
        image = cv2.imread(str(image_path))
        
        # Apply image enhancement
        enhanced_image = enhance_image(image)
        
        # Save the enhanced image to the corresponding folder
        output_path = "enhanced_images/{}/{}".format(disease, image_path.name)
        cv2.imwrite(output_path, enhanced_image)


Cellulitis images: [PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellulitis/BA- cellulitis (125).jpg'), PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellulitis/BA- cellulitis (10).jpg'), PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellulitis/BA- cellulitis (10).jpeg'), PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellulitis/BA- cellulitis (47).jpg'), PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellulitis/BA- cellulitis (51).jpg'), PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellulitis/BA- cellulitis (133).jpg'), PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellulitis/BA- cellulitis (92).jpg'), PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellulitis/BA- cellulitis (109).jpg'), PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images/cellul

In [64]:
cellulitis=list(data_dir.glob('cellulitis/*'))
len(cellulitis)

136

In [65]:
disease_train_label_dic={
    'cellulitis': 0,
    'impetigo': 1,
    'athlete-foot': 2,
    'nail-fungus': 3,
    'ringworm': 4,
    'cutaneous-larva-migrans':5,
    'chickenpox':6,
    'shingles':7,
#    'normal':8,
}

In [66]:
x_train = []
y_train = []

for image_name, image_paths in disease_images_train_dic.items():
    for image_path in image_paths:
        img = cv2.imread(str(image_path))
        if img is None:
            continue
        image_resize=cv2.resize(img,(224,224))
        x_train.append(image_resize)
        y_train.append(disease_train_label_dic[image_name])

In [67]:
x_train[0].shape

(224, 224, 3)

In [68]:
len(disease_images_train_dic['ringworm']),len(disease_images_train_dic['shingles'])

(90, 130)

In [69]:
y_train=np.array(y_train)
x_train=np.array(x_train)
y_train.shape

(925,)

In [70]:
dataset_url='/Users/varunkaleeswaranbalasubramani/Documents/skin-disease-datasaet/test_set copy'

In [71]:
import pathlib
data_dir=pathlib.Path(dataset_url)
data_dir

PosixPath('/Users/varunkaleeswaranbalasubramani/Documents/skin-disease-datasaet/test_set copy')

In [72]:
import cv2
from pathlib import Path


# Define the image enhancement function
def enhance_image(image):
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Convert to YUV color space
    yuv = cv2.cvtColor(blurred, cv2.COLOR_BGR2YUV)
    
    # Apply Laplacian filter for edge enhancement on Y channel
    laplacian = cv2.Laplacian(yuv[:,:,0], cv2.CV_64F)
    
    # Normalize Laplacian for better contrast
    laplacian = cv2.normalize(laplacian, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # Combine Laplacian with original Y channel to enhance edges
    enhanced_y = cv2.addWeighted(yuv[:,:,0], 1.5, laplacian, -0.5, 0)
    
    # Apply histogram equalization to enhance contrast
    enhanced_y = cv2.equalizeHist(enhanced_y)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for better local contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced_y = clahe.apply(enhanced_y)
    
    # Apply unsharp masking for edge and detail enhancement
    blurred = cv2.GaussianBlur(image, (0,0), 3)
    unsharp_mask = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    
    # Merge enhanced Y channel with original U and V channels
    enhanced_yuv = cv2.merge([enhanced_y, yuv[:,:,1], yuv[:,:,2]])
    
    # Convert back to BGR color space
    enhanced_bgr = cv2.cvtColor(enhanced_yuv, cv2.COLOR_YUV2BGR)
    
    return enhanced_bgr

# Define the dictionary containing paths to images for different disease categories
disease_images_test_dic = {
    'cellulitis': list(data_dir.glob('BA- cellulitis/*')),
    'impetigo': list(data_dir.glob('BA-impetigo/*')),
    'athlete-foot': list(data_dir.glob('FU-athlete-foot/*')),
    'nail-fungus': list(data_dir.glob('FU-nail-fungus/*')),
    'ringworm': list(data_dir.glob('FU-ringworm/*')),
    'cutaneous-larva-migrans': list(data_dir.glob('PA-cutaneous-larva-migrans/*')),
    'chickenpox': list(data_dir.glob('VI-chickenpox/*')),
    'shingles': list(data_dir.glob('VI-shingles/*')),
    # 'normal': list(data_dir.glob('normal/*')),
}

# Iterate over each disease category and its corresponding image paths
for disease, image_paths in disease_images_train_dic.items():
    # Create a directory for saving enhanced images if it doesn't exist
    Path("enhanced_images_test/" + disease).mkdir(parents=True, exist_ok=True)
    
    # Iterate over each image path
    for image_path in image_paths:
        # Read the image
        image = cv2.imread(str(image_path))
        
        # Apply image enhancement
        enhanced_image = enhance_image(image)
        
        # Save the enhanced image to the corresponding folder
        output_path = "enhanced_images_test/{}/{}".format(disease, image_path.name)
        cv2.imwrite(output_path, enhanced_image)
dataset_url = '/Users/varunkaleeswaranbalasubramani/Documents/enhanced_images_test'
data_dir=pathlib.Path(dataset_url)
disease_images_test_dic = {
    'cellulitis': list(data_dir.glob('cellulitis/*')),
    'impetigo': list(data_dir.glob('impetigo/*')),
    'athlete-foot': list(data_dir.glob('athlete-foot/*')),
    'nail-fungus': list(data_dir.glob('nail-fungus/*')),
    'ringworm': list(data_dir.glob('ringworm/*')),
    'cutaneous-larva-migrans': list(data_dir.glob('cutaneous-larva-migrans/*')),
    'chickenpox': list(data_dir.glob('chickenpox/*')),
    'shingles': list(data_dir.glob('shingles/*')),
    # 'normal': list(data_dir.glob('normal/*')),
}

In [73]:
x_test = []
y_test = []

for image_name, image_paths in disease_images_test_dic.items():
    for image_path in image_paths:
        img = cv2.imread(str(image_path))
        if img is None:
            continue
        image_resize=cv2.resize(img,(224,224))
        x_test.append(image_resize)
        y_test.append(disease_test_label_dic[image_name])


In [74]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [75]:
x_test.shape

(925, 224, 224, 3)

In [76]:
x_train_scaled=x_train/255
x_test_scaled=x_test/255

In [26]:
x_train_scaled[0]

array([[[0.16078431, 0.21568627, 0.08235294],
        [0.16078431, 0.21568627, 0.08235294],
        [0.16078431, 0.21568627, 0.08235294],
        ...,
        [0.1254902 , 0.19607843, 0.03529412],
        [0.26666667, 0.3372549 , 0.17647059],
        [0.74509804, 0.81568627, 0.65490196]],

       [[0.16078431, 0.21568627, 0.08235294],
        [0.16078431, 0.21568627, 0.08235294],
        [0.16078431, 0.21568627, 0.08235294],
        ...,
        [0.13333333, 0.20392157, 0.04313725],
        [0.27058824, 0.34117647, 0.18039216],
        [0.72941176, 0.8       , 0.63921569]],

       [[0.16078431, 0.21568627, 0.08235294],
        [0.16078431, 0.21568627, 0.08235294],
        [0.16078431, 0.21568627, 0.08235294],
        ...,
        [0.14117647, 0.21176471, 0.05098039],
        [0.26666667, 0.3372549 , 0.17647059],
        [0.69411765, 0.76470588, 0.60392157]],

       ...,

       [[0.43529412, 0.5254902 , 0.72156863],
        [0.42352941, 0.51372549, 0.70980392],
        [0.40392157, 0

In [77]:
import tensorflow_hub as hub

In [78]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224, 224, 3),
                                         trainable=False)  

In [79]:
x_train_scaled.shape

(925, 224, 224, 3)

In [80]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(9, activation='softmax')  # Adjust the number of units according to your number of classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model architecture
model.summary()





Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     9,437,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │         4,617 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,683,145 (36.94 MB)

 Trainable params: 9,683,145 (36.94 MB)

 Non-trainable params: 0 (0.00 B)

In [81]:
# Train the model
model.fit(x_train_scaled, y_train, epochs=500)

Epoch 1/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 309ms/step - accuracy: 0.1143 - loss: 2.2664
Epoch 2/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 303ms/step - accuracy: 0.4048 - loss: 1.6305
Epoch 3/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 307ms/step - accuracy: 0.6243 - loss: 1.0876
Epoch 4/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 307ms/step - accuracy: 0.7429 - loss: 0.7610
Epoch 5/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 303ms/step - accuracy: 0.8736 - loss: 0.3560
Epoch 6/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 306ms/step - accuracy: 0.9266 - loss: 0.2309
Epoch 7/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 305ms/step - accuracy: 0.9632 - loss: 0.1397
Epoch 8/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 307ms/step - accuracy: 0.9940 - loss: 0.0360
Epoch 9/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 304ms/step - accuracy: 1.0000 - loss: 0.0064
Epoch 10/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 307ms/step - accuracy: 1.0000 - loss: 0.0019
Epoch 11/500
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 308ms/step - accuracy: 1.0000 - loss: 6.1339e-04
Epoch 12/500
29/29 ━━━━━━━━━━━━━━━━━

In [82]:
x_test_scaled.shape

(925, 224, 224, 3)

In [83]:
model.evaluate(x_test_scaled,y_test)

29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.9898 - loss: 0.0803


[0.2957117259502411, 0.9599999785423279]

In [85]:
model.save('derment_ai.keras')